## Imports

In [1]:
import numpy as np
import pandas as pd

## Read data

### 1b. Filled ISH

In [15]:
ish = pd.read_csv("/beegfs/home/pmatyskova/project/ish_data_cor_hypnoSFO_filled1_0.csv") 

In [16]:
ish_full_genenames = np.load("/beegfs/home/pmatyskova/project/ish_data_cor_genenames.npy")

ish = ish.drop(['Unnamed: 0'], axis=1)
ish.columns = ish_full_genenames
e_matrix_full = np.transpose(ish)
e_matrix = e_matrix_full[~e_matrix_full.index.duplicated(keep=False)]

### 2. Single cell RNA seq

In [17]:
hyp = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/data/petra_data/hypoMap_avg_signatures/hypomap_avg_signatures_C185_rna.txt', sep = "\t")
hyp.index = hyp.iloc[:,0]
c_mat_full = hyp.drop(['Unnamed: 0'], axis=1)

genes_to_exclude = pd.read_csv('/beegfs/scratch/bruening_scratch/lsteuernagel/projects/analysis_projects/volumetric_analysis/genes_to_exclude.csv',
                               header = None)
c_mat_full = c_mat_full[~c_mat_full.index.isin(genes_to_exclude.iloc[:,0])]

### 3. The rest of the import pipeline

In [18]:
e_matrix = e_matrix[e_matrix.index.isin(c_mat_full.index)] #keep the common ones
c_matrix = c_mat_full[c_mat_full.index.isin(e_matrix.index)]

match_index = [list(c_matrix.index).index(i) for i in list(e_matrix.index)] #find indeces to reorder hyp to ish
c_matrix = c_matrix.iloc[match_index,]

In [20]:
gene_annot = c_matrix.index #save gene names to know what genes selected
cellt_annot = c_matrix.columns #save cellt names to annotate np array later

c_nd = c_matrix.to_numpy()
e_nd = e_matrix.to_numpy()

In [21]:
dup_labels = np.unique(e_matrix_full[e_matrix_full.index.duplicated(keep=False)].index)
dup_labels = dup_labels[np.isin(dup_labels, c_mat_full.index)]

## Run minimum residuals

In [22]:
for i in range(len(dup_labels)): 
    print(i)
    c_gene = c_mat_full[c_mat_full.index == dup_labels[i]]
    e_gene = e_matrix_full[e_matrix_full.index == dup_labels[i]]
    
    col_cv = np.sqrt(e_gene.var(axis=1))/e_gene.mean(axis=1)
    
    max_ind = np.argmax(col_cv)
    e_nd = np.vstack((e_nd, e_gene.iloc[max_ind, ]))
    c_nd = np.vstack((c_nd, c_gene))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229


In [25]:
geneselect = np.append(gene_annot,dup_labels)

np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_c_hm185cor_coefvar_filled0", arr = c_nd)
np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_e_hm185cor_coefvar_filled0", arr = e_nd)
np.save(file = "/beegfs/home/pmatyskova/project/ishfstep_geneset_hm185cor_coefvar_filled0", arr = geneselect)